In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/MIR-Phase3-2/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/content/drive/MyDrive/MIR-Phase3-2/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])



In [ ]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [ ]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

def genre_recommendations(title, n=10):
    if title not in indices:
        raise ValueError(f"Title '{title}' not found in the dataset.")

    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]

    recommended_indices = [index for index, score in sorted_scores]

    return titles.iloc[recommended_indices]


In [ ]:
# Example
genre_recommendations('GoldenEye (1995)')

345     Clear and Present Danger (1994)
543           Surviving the Game (1994)
724                    Rock, The (1996)
788                     Daylight (1996)
825               Chain Reaction (1996)
978                 Maximum Risk (1996)
1467                    Anaconda (1997)
1513                     Con Air (1997)
1693                   Firestorm (1998)
3686          Perfect Storm, The (2000)
Name: title, dtype: object


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.merge(ratings, movies, on='movie_id')

In [ ]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [ ]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

In [ ]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [ ]:
def create_user_genre_matrix(data):
    user_genre_matrix = data.pivot_table(index='user_id', columns='genres', aggfunc='size', fill_value=0)
    return user_genre_matrix.div(user_genre_matrix.sum(axis=1), axis=0)


In [ ]:
def calculate_user_similarity(user_genre_matrix):
    user_similarity = cosine_similarity(user_genre_matrix)
    return pd.DataFrame(user_similarity, index=user_genre_matrix.index, columns=user_genre_matrix.index)

In [ ]:
def get_top_n_similar_users(user_id, user_similarity_df, n=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).drop(user_id).index.tolist()
    return similar_users[:n]

In [ ]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    user_genre_matrix = create_user_genre_matrix(data)
    user_similarity_matrix = calculate_user_similarity(user_genre_matrix)

    similar_users = get_top_n_similar_users(user_id, user_similarity_matrix, n_similar_users)
    similar_users_ratings = data[data['user_id'].isin(similar_users)]
    target_user_seen_movies = data[data['user_id'] == user_id]['movie_id'].unique()
    similar_users_unseen_movies = similar_users_ratings[~similar_users_ratings['movie_id'].isin(target_user_seen_movies)]
    top_movie_ids = similar_users_unseen_movies['movie_id'].value_counts().head(n_recommendations).index.tolist()
    top_recommended_movies = pd.merge(movies, pd.DataFrame(top_movie_ids, columns=['movie_id']), on='movie_id')[['title', 'genres']]

    return top_recommended_movies

In [ ]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [ ]:
recommended_movies

,title,genres
0,"Silence of the Lambs, The (1991)","['Drama', 'Thriller']"
1,Pinocchio (1940),"['Animation', ""Children's""]"
2,Alice in Wonderland (1951),"['Animation', ""Children's"", 'Musical']"
3,Amadeus (1984),['Drama']
4,Akira (1988),"['Adventure', 'Animation', 'Sci-Fi', 'Thriller']"
5,Lady and the Tramp (1955),"['Animation', ""Children's"", 'Comedy', 'Musical..."
6,Sleeping Beauty (1959),"['Animation', ""Children's"", 'Musical']"
7,American Beauty (1999),"['Comedy', 'Drama']"
8,Magnolia (1999),['Drama']
9,High Fidelity (2000),['Comedy']


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [ ]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [ ]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# calculate rating matrix

ratings_array = Ratings.values
mean_user_ratings = np.mean(ratings_array, axis=1)
demeaned_ratings = ratings_array - mean_user_ratings[:, np.newaxis]

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(demeaned_ratings, k=50)
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + mean_user_ratings[:, np.newaxis]

In [ ]:
preds = pd.DataFrame(all_user_predicted_ratings, columns=Ratings.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,...,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,...,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,...,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,...,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,...,0.110151,0.046010,0.006934,-0.015940,-0.050080,-0.052539,0.507189,0.033830,0.125706,0.199244


In [ ]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):

    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False)

    user_data = original_ratings[original_ratings.user_id == (userID)]

    # merge movies data with user_data
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False))


    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending highest {num_recommendations} predicted ratings movies not already rated.')

    unwatched_movies = movies[~movies['movie_id'].isin(user_full['movie_id'])]

    recommendations = (unwatched_movies
                          .merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left', on='movie_id')
                          .rename(columns={sorted_user_predictions.name: 'Predictions'})
                          .sort_values('Predictions', ascending=False)
                          .iloc[:num_recommendations, :-1])

    return user_full, recommendations

In [ ]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [ ]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [ ]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),['Drama']
285,4375,175,5,4374,174,Kids (1995),['Drama']
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)","['Action', 'Drama', 'Romance', 'Thriller']"
65,4375,3809,5,4374,3808,What About Bob? (1991),['Comedy']
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)","['Drama', 'Thriller']"
270,4375,150,5,4374,149,Apollo 13 (1995),['Drama']
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",['Horror']
125,4375,349,5,4374,348,Clear and Present Danger (1994),"['Action', 'Adventure', 'Thriller']"
60,4375,288,5,4374,287,Natural Born Killers (1994),"['Action', 'Thriller']"
141,4375,47,5,4374,46,Seven (Se7en) (1995),"['Crime', 'Thriller']"


#### Evaluation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2)

train_ratings = train_data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
R_train = train_ratings.values
user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train = svds(Ratings_demeaned_train, k=50)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train) + user_ratings_mean_train.reshape(-1, 1)
preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)

def predict_rating(user_id, movie_id):
    user_id = user_id - 1
    try:
      pred = preds_train.iloc[user_id, movie_id - 1]
    except IndexError:
      pred = np.nan
    return pred

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 3.496939333229388


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error

class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, index):
        rating_row = self.ratings.iloc[index]
        return (
            torch.tensor(rating_row['user_emb_id'], dtype=torch.long),
            torch.tensor(rating_row['movie_emb_id'], dtype=torch.long),
            torch.tensor(rating_row['rating'], dtype=torch.float)
        )

class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, user, movie):
        user_emb = self.user_embedding(user)
        movie_emb = self.movie_embedding(movie)
        x = torch.cat([user_emb, movie_emb], dim=-1)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.sigmoid(self.fc4(x)) * 4 + 1
        return x

def train_model(model, dataloader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for user_batch, movie_batch, rating_batch in dataloader:
            optimizer.zero_grad()
            predictions = model(user_batch, movie_batch).squeeze()
            loss = criterion(predictions, rating_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(dataloader)
        print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss}')

        checkpoint_path = f'model_epoch_{epoch + 1}.pth'
        torch.save(model.state_dict(), checkpoint_path)

def evaluate_model(model, dataloader):
    model.eval()
    all_predictions = []
    all_actual_ratings = []
    with torch.no_grad():
        for user_batch, movie_batch, rating_batch in dataloader:
            predictions = model(user_batch, movie_batch).squeeze()
            all_predictions.extend(predictions.tolist())
            all_actual_ratings.extend(rating_batch.tolist())

    rmse = mean_squared_error(all_actual_ratings, all_predictions, squared=False)
    print(f'RMSE: {rmse}')
    return rmse

num_users = ratings['user_emb_id'].max() + 1
num_movies = ratings['movie_emb_id'].max() + 1

ratings_dataset = RatingsDataset(ratings)
ratings_dataloader = DataLoader(ratings_dataset, batch_size=64, shuffle=True)

model = RecommenderNet(num_users, num_movies)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


train_model(model, ratings_dataloader, criterion, optimizer, num_epochs=5)

evaluate_model(model, ratings_dataloader)


Epoch 1/5, Average Loss: 0.942580288973153
Epoch 2/5, Average Loss: 0.8354936689262359
Epoch 3/5, Average Loss: 0.8113635556964348
Epoch 4/5, Average Loss: 0.7953360783683109
Epoch 5/5, Average Loss: 0.7802905769283006
RMSE: 0.8628836093289058


0.8628836093289058

In [ ]:
# Example

sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

Sample predictions for user ID 1:
Movie ID: 1192, Predicted Rating: 4.3314666748046875, Actual Rating: 5.0
Movie ID: 660, Predicted Rating: 3.6167073249816895, Actual Rating: 3.0
Movie ID: 913, Predicted Rating: 4.3764448165893555, Actual Rating: 3.0
Movie ID: 3407, Predicted Rating: 4.231060028076172, Actual Rating: 4.0
Movie ID: 2354, Predicted Rating: 3.997720956802368, Actual Rating: 5.0
Movie ID: 1196, Predicted Rating: 4.465364456176758, Actual Rating: 3.0
Movie ID: 1286, Predicted Rating: 4.453712463378906, Actual Rating: 5.0
Movie ID: 2803, Predicted Rating: 4.466440200805664, Actual Rating: 5.0
Movie ID: 593, Predicted Rating: 3.958430528640747, Actual Rating: 4.0
Movie ID: 918, Predicted Rating: 4.594278812408447, Actual Rating: 4.0
Movie ID: 594, Predicted Rating: 4.244167327880859, Actual Rating: 5.0
Movie ID: 937, Predicted Rating: 4.05381965637207, Actual Rating: 4.0
Movie ID: 2397, Predicted Rating: 4.2929887771606445, Actual Rating: 4.0
Movie ID: 2917, Predicted Rating:

In [ ]:
def predict_unrated_movies(user_id, model, ratings, movies, top_n=10):
    user_embedding_id = ratings[ratings['user_id'] == user_id]['user_emb_id'].unique()[0]

    user_rated_movies = ratings[ratings['user_id'] == user_id]
    unrated_movies = ratings[~ratings['movie_id'].isin(user_rated_movies['movie_id'])][['movie_id']].drop_duplicates()

    user_embedding_tensor = torch.tensor([user_embedding_id] * len(unrated_movies), dtype=torch.long)
    movie_embedding_tensor = torch.tensor(
        [ratings[ratings['movie_id'] == movie_id]['movie_emb_id'].values[0] for movie_id in unrated_movies['movie_id']],
        dtype=torch.long
    )

    with torch.no_grad():
        predicted_ratings = model(user_embedding_tensor, movie_embedding_tensor).squeeze().tolist()

    unrated_movies['predicted_rating'] = predicted_ratings

    recommended_movies = unrated_movies.merge(movies, on='movie_id')

    top_recommendations = recommended_movies.sort_values(by='predicted_rating', ascending=False).head(top_n)

    return top_recommendations[['movie_id', 'title', 'genres', 'predicted_rating']]


user_id = 1
top_predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(top_predictions)


      movie_id                                              title  \
74        1198                     Raiders of the Lost Ark (1981)   
1039      2019  Seven Samurai (The Magnificent Seven) (Shichin...   
640        953                       It's a Wonderful Life (1946)   
114        318                   Shawshank Redemption, The (1994)   
183       1250               Bridge on the River Kwai, The (1957)   
576        904                                 Rear Window (1954)   
1396       922      Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)   
616        858                              Godfather, The (1972)   
482       1148                         Wrong Trousers, The (1993)   
206         50                         Usual Suspects, The (1995)   

                            genres  predicted_rating  
74         ['Action', 'Adventure']          4.762063  
1039           ['Action', 'Drama']          4.705402  
640                      ['Drama']          4.701746  
114                 

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [63]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

countries = [
    "United States", "China", "India", "Japan", "Brazil",
    "Russia", "Indonesia", "Germany", "United Kingdom", "France"
]

file_path = '/content/drive/MyDrive/MIR-Phase3-2/gmm-dataset.csv'

df = pd.read_csv(file_path)
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(df.drop(columns=['URL']).values)

log_likelihood = gmm.score(df[countries]) * df.shape[0]
bic = gmm.bic(df[countries])
aic = gmm.aic(df[countries])

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

Log-Likelihood: -6947.5919428507905
BIC: 14802.402412341235
AIC: 14289.183885701581


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
